In [4]:
import os

os.chdir("/Users/shivatmaksharma/D/Uni Work/Advanced ML/Assignment 2/SalesDynamite")

In [5]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline
from xgboost import XGBRegressor
import numpy as np
from src.data.data_loader_f import DataLoaderDebug
from src.features.data_transformer_f import DataTransformerDebug
from src.models.predict_model import PredictiveModeling
from src.models.forecasting_model import ForecastingModel
from joblib import dump

In [6]:
# Main Pipeline
forecasting_pipeline = Pipeline([
    ('loader', DataLoaderDebug('data/raw/sales_train.csv', 
                               'data/raw/calendar.csv', 
                               'data/raw/calendar_events.csv', 
                               'data/raw/items_weekly_sell_prices.csv')),
    ('transformer', DataTransformerDebug()),
    ('forecaster', ForecastingModel())
])

# Execute the pipeline
X_transformed = forecasting_pipeline.fit_transform(X=None, y=None)

# Extract the 'date', 'event_type', and 'revenue' columns after transformations for evaluation
df = X_transformed[['date', 'event_type', 'revenue']].groupby('date').sum().reset_index()

# Splitting the data for evaluation
train = df[df['date'] < '2015-04-11']  # Training data excluding the last 7 days
test = df[df['date'] >= '2015-04-11']  # Test data for the last 7 days

print(df.tail())
print("Length of train:", len(train))
print("Length of test:", len(test))

# Train the model
forecasting_model = ForecastingModel()
forecasting_model.fit(train)

# Save the trained forecasting model
#dump(forecasting_model, 'models/forecasting/forecasting_model.joblib')

# Evaluate the model on the test data
evaluation_metrics = forecasting_model.evaluate(test, len(test))
print("Evaluation Metrics:", evaluation_metrics)

# Define the start date from which you want to forecast the next 7 days
start_date = '2015-06-18'  # Adjust this as needed

# Retrieve the forecasted values for the next 7 days from the specified start date
forecast = forecasting_pipeline.named_steps['forecaster'].predict(start_date=start_date)

# Display the forecast
print("\nForecast:")
print(forecast)



Data after melt:
                                      id        item_id    dept_id   cat_id  \
0         HOBBIES_1_001_CA_1_evaluation  HOBBIES_1_001  HOBBIES_1  HOBBIES   
1         HOBBIES_1_002_CA_1_evaluation  HOBBIES_1_002  HOBBIES_1  HOBBIES   
2         HOBBIES_1_003_CA_1_evaluation  HOBBIES_1_003  HOBBIES_1  HOBBIES   
3         HOBBIES_1_004_CA_1_evaluation  HOBBIES_1_004  HOBBIES_1  HOBBIES   
4         HOBBIES_1_005_CA_1_evaluation  HOBBIES_1_005  HOBBIES_1  HOBBIES   
...                                 ...            ...        ...      ...   
46985085    FOODS_3_823_WI_3_evaluation    FOODS_3_823    FOODS_3    FOODS   
46985086    FOODS_3_824_WI_3_evaluation    FOODS_3_824    FOODS_3    FOODS   
46985087    FOODS_3_825_WI_3_evaluation    FOODS_3_825    FOODS_3    FOODS   
46985088    FOODS_3_826_WI_3_evaluation    FOODS_3_826    FOODS_3    FOODS   
46985089    FOODS_3_827_WI_3_evaluation    FOODS_3_827    FOODS_3    FOODS   

         store_id state_id       d  sales  


00:22:19 - cmdstanpy - INFO - Chain [1] start processing
00:22:19 - cmdstanpy - INFO - Chain [1] done processing
00:22:20 - cmdstanpy - INFO - Chain [1] start processing
00:22:20 - cmdstanpy - INFO - Chain [1] done processing


           date  event_type    revenue
1536 2015-04-14      -30490  102896.92
1537 2015-04-15      -30490  106913.88
1538 2015-04-16      -30490   98373.49
1539 2015-04-17      -30490  103030.58
1540 2015-04-18      -30490  132625.89
Length of train: 1533
Length of test: 8
Evaluation Metrics: {'MAE': 13157.11864609534, 'MSE': 370070334.04536206, 'RMSE': 19237.212221248745, 'MAPE': 11.096095237491392}

Forecast:
          ds          yhat    yhat_lower    yhat_upper
0 2015-06-19  18259.806192   6749.085908  30359.985089
1 2015-06-20  38447.888740  25968.804236  49431.290245
2 2015-06-21  38376.068478  27128.319231  50219.330324
3 2015-06-22  14948.659834   3924.686560  27407.268219
4 2015-06-23   7649.182949  -3431.082169  19834.792781
5 2015-06-24   6706.565231  -5234.783182  17642.759270
6 2015-06-25   6848.248047  -4582.594534  18686.651526
